###Post processes Gaul Locations with different buffer sizes after manually adding them in QGIS
##Harmonizes country names
##Selects the best match for multiple rows with the same DHSID and differing country names

In [1]:
import difflib
import numpy as np
import pandas as pd

In [2]:
file = '/mnt/datadisk/data/Projects/water/inputs/final_locations_gaul.csv'
df = pd.read_csv(file)
df


,country,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,...,adm0_code_,adm0_name_,Shape_Ar_1,adm2_cod_1,adm2_nam_1,adm1_cod_1,adm1_nam_1,adm0_cod_1,adm0_nam_1,Shape_Ar_2
0,Burundi,BU201200000070,BU,2012.0,70.0,BY,NaN,NaN,NaN,NaN,...,43.0,Burundi,0.013315,40621.0,Ryansoro,40548.0,Gitega,43.0,Burundi,0.013315
1,Burundi,BU201600000414,BU,2016.0,414.0,BY,NaN,NaN,NaN,NaN,...,43.0,Burundi,0.013315,40621.0,Ryansoro,40548.0,Gitega,43.0,Burundi,0.013315
2,Burundi,BU201600000247,BU,2016.0,247.0,BY,NaN,NaN,NaN,NaN,...,43.0,Burundi,0.013315,40621.0,Ryansoro,40548.0,Gitega,43.0,Burundi,0.013315
3,Burundi,BU201600000329,BU,2016.0,329.0,BY,NaN,NaN,NaN,NaN,...,43.0,Burundi,0.013315,40621.0,Ryansoro,40548.0,Gitega,43.0,Burundi,0.013315
4,Burundi,BU201600000309,BU,2016.0,309.0,BY,NaN,NaN,NaN,NaN,...,43.0,Burundi,0.013315,40621.0,Ryansoro,40548.0,Gitega,43.0,Burundi,0.013315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250499,Mali,ML201800000319,ML,2018.0,319.0,ML,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250500,Mali,ML201800000320,ML,2018.0,320.0,ML,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250501,Mozambique,MZ201100000610,MZ,2011.0,610.0,MZ,MZ11,Maputo (city),Maputo (city),MOZ006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250502,Nigeria,NG200800000773,NG,2008.0,773.0,NG,NI50,RIVERS,RIVERS,NGA033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
countries = df["country"].unique()
adm0_names = df["adm0_name"].unique()
unmatched_countries = [c for c in countries if c not in adm0_names]
unmatched_adm0_names = [c for c in adm0_names if c not in countries]
print(unmatched_countries)
print(unmatched_adm0_names)
country_rename = {'Congo Democratic Republic': 'Democratic Republic of the Congo', 'Tanzania': 'United Republic of Tanzania', "Cote d'Ivoire": "Côte d'Ivoire"}
df["country"] = df["country"].replace(country_rename)

['Congo Democratic Republic', 'Tanzania', "Cote d'Ivoire"]
['Democratic Republic of the Congo', 'United Republic of Tanzania', 'South Sudan', 'Sudan', nan, 'Somalia', 'Western Sahara', 'Guinea-Bissau', 'Mauritania', "Côte d'Ivoire"]


In [4]:
#combine multiple columns if values are missing
for adm in ['adm0_name', 'adm1_name', 'adm2_name']:
    #select rows where admX_name is missing and fill in with admX_name_
    df.loc[df[adm].isnull(), adm] = df.loc[df[adm].isnull(), adm+'_']
    # missing_adm = df[df[adm].isnull()]

#compare 2 columns for overlapping values
#not one single match...
print(len(df))
print(len(df[df['ADM1NAME'] != df['adm1_name']]))
print(len(df[df['ADM1NAME'] == df['adm1_name']]))
print(len(df[df['DHSREGNA'] != df['adm1_name']]))
print(len(df[df['DHSREGNA'] == df['adm1_name']]))
df['ADM1NAME'] = df['ADM1NAME'].fillna('').astype(str)
df['adm1_name'] = df['adm1_name'].fillna('').astype(str)
df['DHSREGNA'] = df['DHSREGNA'].fillna('').astype(str)

# difflib.get_close_matches(df['adm1_name'], df['ADM1NAME'])

250504
175294
75210
210093
40411


In [5]:
def select_row(group):
    """Cleanup multiple rows with the same dhsid but differing country names"""
    # Check if any row in the group has country == adm0_name
    matching_rows = group[group['country'] == group['adm0_name']]
    if not matching_rows.empty:
        #compare adm1_name and ADM1NAME
        if len(matching_rows) > 1:
            best_match_l = []
            for _, row in matching_rows.iterrows():
                if row['ADM1NAME'] in difflib.get_close_matches(row['adm1_name'], matching_rows['ADM1NAME']):
                    best_match_l.append(row)
                if row['DHSREGNA'] in difflib.get_close_matches(row['adm1_name'], matching_rows['DHSREGNA']):
                    best_match_l.append(row)
                if len(best_match_l) >= 2:
                    # match_adm = [row['adm1_name'] for row in best_match_l]
                    match_values_l = [best_match_l[0]['ADM1NAME'], best_match_l[1]['DHSREGNA']]
                    for row1, match in zip(best_match_l, match_values_l):
                        if match in difflib.get_close_matches(row1['adm1_name'], match_values_l):
                            best_match_l.append(row1)
                            # update match count
                            select_row.match_count += 1
                            return row1
            else:
                # If no close match, return the first row in the group
                return matching_rows.iloc[0]
        else:
            return matching_rows.iloc[0]
    else:
        # Otherwise, return the first row in the group
        return group.iloc[0]

# initialize match count
select_row.match_count = 0
result_df = df.groupby('DHSID').apply(select_row)

print("Number of matches found:", select_row.match_count)
print(len(result_df))

Number of matches found: 24526
70756


In [8]:
#drop columns
cols = ['adm0_name_', 'adm1_name_', 'adm2_name_', 'adm0_code_', 'adm1_code_', 'adm2_code_' ]
final_df = result_df.drop(columns=cols)
final_df

,country,DHSID,DHSCC,DHSYEAR,DHSCLUST,CCFIPS,ADM1FIPS,ADM1FIPSNA,ADM1SALBNA,ADM1SALBCO,...,adm0_name,Shape_Area,Shape_Ar_1,adm2_cod_1,adm2_nam_1,adm1_cod_1,adm1_nam_1,adm0_cod_1,adm0_nam_1,Shape_Ar_2
DHSID,,,,,,,,,,,,,,,,,,,,,
AO200600000001,Angola,AO200600000001,AO,2006.0,1.0,AO,NaN,NaN,NaN,NaN,...,Angola,0.289515,0.289515,4222.0,Lobito,399.0,Benguela,8.0,Angola,0.289515
AO200600000002,Angola,AO200600000002,AO,2006.0,2.0,AO,NaN,NaN,NaN,NaN,...,Angola,0.289515,0.289515,4222.0,Lobito,399.0,Benguela,8.0,Angola,0.289515
AO200600000003,Angola,AO200600000003,AO,2006.0,3.0,AO,NaN,NaN,NaN,NaN,...,Angola,0.196329,0.196329,4216.0,Benguela,399.0,Benguela,8.0,Angola,0.196329
AO200600000004,Angola,AO200600000004,AO,2006.0,4.0,AO,NaN,NaN,NaN,NaN,...,Angola,0.329282,0.329282,4254.0,Seles,403.0,Cuanza Sul,8.0,Angola,0.330459
AO200600000005,Angola,AO200600000005,AO,2006.0,5.0,AO,NaN,NaN,NaN,NaN,...,Angola,0.569600,0.569600,4208.0,Dande,398.0,Bengo,8.0,Angola,0.569600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000396,Zimbabwe,ZW201500000396,ZW,2015.0,396.0,ZW,NaN,NaN,NaN,NaN,...,Zimbabwe,0.081806,0.212495,33067.0,Goromonzi,69550.0,Mashonaland East,271.0,Zimbabwe,0.212495
ZW201500000397,Zimbabwe,ZW201500000397,ZW,2015.0,397.0,ZW,NaN,NaN,NaN,NaN,...,Zimbabwe,0.671006,0.671006,33056.0,Makoni,3437.0,Manicaland,271.0,Zimbabwe,0.671006
ZW201500000398,Zimbabwe,ZW201500000398,ZW,2015.0,398.0,ZW,NaN,NaN,NaN,NaN,...,Zimbabwe,1.681446,1.681446,33076.0,Hurungwe,3440.0,Mashonaland West,271.0,Zimbabwe,1.681446


In [9]:
final_df[final_df["country"] != final_df["adm0_name"]][["adm0_name", "country", "DHSID", "adm1_name", "ADM1NAME", "adm2_name"]]

,adm0_name,country,DHSID,adm1_name,ADM1NAME,adm2_name
DHSID,,,,,,
BJ199651402023,NaN,Benin,BJ199651402023,,OUEME,NaN
BJ201200000126,NaN,Benin,BJ201200000126,,Atlantique,NaN
BJ201200000128,NaN,Benin,BJ201200000128,,Atlantique,NaN
BJ201200000500,NaN,Benin,BJ201200000500,,Mono,NaN
BJ201200000569,NaN,Benin,BJ201200000569,,Oueme,NaN
...,...,...,...,...,...,...
ZM201800000179,NaN,Zambia,ZM201800000179,,Luapula,NaN
ZM201800000387,Democratic Republic of the Congo,Zambia,ZM201800000387,Katanga,Luapula,Haut-Katanga
ZW200500000450,Mozambique,Zimbabwe,ZW200500000450,Tete,Mashonaland Central,Magoe


In [10]:
#write country into adm0_name if adm0_name is missing or country differs from adm0_name
final_df.loc[final_df["country"] != final_df["adm0_name"], "adm0_name"] = final_df.loc[final_df["country"] != final_df["adm0_name"], "country"]

In [11]:
final_df.describe()

,DHSYEAR,DHSCLUST,ADM1DHS,DHSREGCO,LATNUM,LONGNUM,ALT_GPS,ALT_DEM,adm2_code,adm1_code,adm0_code,Shape_Area,Shape_Ar_1,adm2_cod_1,adm1_cod_1,adm0_cod_1,Shape_Ar_2
count,70756.000000,7.075600e+04,70756.000000,70756.000000,70756.000000,70756.000000,70756.000000,70756.000000,70537.000000,70537.000000,70537.000000,70537.000000,70721.000000,70732.000000,70732.000000,70732.000000,70732.000000
mean,2010.405817,3.839435e+05,1583.887275,33.393267,3.536423,18.061645,7385.895991,827.449021,63013.976580,25257.370486,4616.822590,0.539649,0.542024,62128.117670,25301.761508,4613.572556,0.570762
std,7.602490,2.306894e+06,3623.487541,491.277119,14.745476,18.725756,4215.088461,1493.371782,56143.363957,29295.233661,12703.917535,1.486384,1.482488,55796.468123,29362.900877,12699.152143,1.491075
min,1986.000000,1.000000e+00,0.000000,0.000000,-34.463232,-17.503485,-306.000000,-92.000000,1379.000000,398.000000,6.000000,0.000055,0.000100,1379.000000,398.000000,8.000000,0.000101
25%,2005.000000,1.210000e+02,4.000000,2.000000,-4.306040,0.318297,1468.052500,66.000000,21987.000000,1824.000000,94.000000,0.033509,0.034948,21977.000000,1828.000000,94.000000,0.041683
50%,2012.000000,2.580000e+02,9.000000,4.000000,6.052601,28.241568,9999.000000,356.000000,42215.000000,3427.000000,169.000000,0.161193,0.162840,42175.000000,3426.500000,170.000000,0.178381
75%,2016.000000,5.050000e+02,37.000000,8.000000,12.126208,33.239190,9999.000000,1181.000000,66017.000000,48368.000000,243.000000,0.533860,0.539800,65959.000000,48369.000000,243.000000,0.568114
max,2022.000000,6.150806e+07,10228.000000,9999.000000,35.858894,50.457728,9999.000000,9999.000000,191425.000000,190430.000000,40765.000000,28.762152,28.762152,191424.000000,190431.000000,40765.000000,28.762152


In [12]:
final_df.to_csv('/mnt/datadisk/data/Projects/water/inputs/all_locations_gaul_final2.csv', index=False)
